In [1]:

import numpy as np
import argparse
import tqdm
import pickle
import pandas as pd
import os
import datetime
import warnings
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from utility.utility import *
from utility.training import callback
import tensorflow as tf

import configparser

# turn off system warning 
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


c:\Users\yayay\.virtualenvs\StockPricePrediction-MviMxm2U\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### take data from ml-lstm

In [15]:

data = pickle.load(open('data.pk', 'rb'))

c:\Users\yayay\.virtualenvs\StockPricePrediction-MviMxm2U\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:

data.stockPrice   
priceSet=data.stockPrice
display(priceSet.head(3)) 
# display(targetSet.iloc[:].head(1))

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,-0.531974,-0.979487,-0.867862,-0.604503,-0.704443,-0.611049,-0.404988,-0.884426,0.705216,-0.647578,...,-0.195646,-0.979320,-0.882948,0.540395,-0.976658,-0.271402,0.044118,-0.879489,1.000000,-0.871450
1,-0.582389,-0.974358,-0.853058,-0.610290,-0.703443,-0.563007,-0.421337,-0.887295,0.716429,-0.609040,...,-0.232431,-0.980996,-0.874393,0.480043,-0.961096,-0.276113,-0.073529,-0.874491,0.966781,-0.872209
2,-0.597094,-0.993323,-0.885761,-0.634593,-0.709441,-0.688526,-0.533492,-0.885656,0.660312,-0.628830,...,-0.210360,-0.991057,-0.873289,0.200326,-0.943589,-0.290245,-0.078431,-0.884487,0.847192,-0.871450


In [17]:
# data.trainSet[0]['target_history']
data.validSet[0]['target_history']

AttributeError: 'StockData' object has no attribute 'validSet'

### Process

In [12]:

config = configparser.ConfigParser()
config.read('config.ini')

# main(config )

# def main(config):
start_time = datetime.datetime.now()
mode='train'
model_type='dnn'
# basic setting
make_dir(config)
maybe_make_dir(f'logs/train/{model_type}')
datetime_prefix = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# reading data
############################## original
df = read_data(config, mode)
X, y = label(df)
X_scaler, y_scaler = scaler(X, y, config, mode)
X_scaler, y_scaler = training_window(X_scaler, y_scaler , config, model_type)
############################## original
X, y = 

model = load_model(X_scaler.shape, model_type)
# print(args.mode) 

if mode == 'train':
    X_train, y_train, X_valid, y_valid, split = split_valid(X_scaler, y_scaler, config)
    history = model.fit(
        X_train, y_train, 
        epochs=int(config['MODEL']['epoch']),
        verbose = 1,
        batch_size=4,
        validation_data=(X_valid, y_valid), 
        # callbacks = callback(config, args, datetime_prefix)
    )
    y_pred = model.predict(X_valid)

    # if bool(config["STOCK"]["scale"]):
    #     y_pred = inverse_predict(y_pred, config)

    model_setting = config['MODEL']
    model.save_weights(f'model/{model_type}/{datetime_prefix}_{config["STOCK"]["stock"]}_e{model_setting["epoch"]}_s{model_setting["slide"]}')

    # score = model.evaluate(X_valid, y_valid, verbose=0)
    valid_mse = mean_squared_error(np.array(y[int(config['MODEL']['slide'])+split:]), y_pred, squared=False)   
    # print(pd.DataFrame((np.array(y[int(config['MODEL']['slide'])+split:]),y_pred_inverse)))
    print('mse',valid_mse)
    pd.DataFrame(history.history).to_csv(f'logs/csv_logger/{model_type}/{datetime_prefix}_{valid_mse}.csv')

if mode == 'test':
    model.load_weights(f'model/{model_type}/{config["MODEL_WEIGHTS"][model_type]}')
    y_pred = model.predict(X_scaler)
    y_inverse = inverse_predict(y_pred, config)
    mse = mean_squared_error(y[int(config['MODEL']['slide']):], y_inverse, squared=False)   
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 4), dpi=100)

    sns.lineplot(data = y[int(config['MODEL']['slide']):], ax=ax, color='r',label='y_t')
    sns.lineplot(data = y_inverse, ax=ax,color='c',label='r')
    plt.show()
    print(model_type,'mse',mse)

# parser = argparse.ArgumentParser()
# parser.add_argument('-m', '--mode', type=str, default = 'train', required=False, help='either "train" or "test"')
# # parser.add_argument('-e', '--episode', type=int, default=10, help='number of episode to run')
# parser.add_argument('-t', '--model_type', type=str, default='dnn', required=False, help='"dnn", "conv1d", "conv2d", "lstm" or "transformer"')
# # parser.add_argument('-s', '--stock', type=str, required=False, default='TWII', help='stock index')
# parser.add_argument('-w', '--weight', type=str, required=False, help='stock portfolios')
# args = parser.parse_args()

# config = configparser.ConfigParser()
# config.read('config.ini')

# main(config )
    

data\TWII_train.csv

scale

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              7168      
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)            